# AudioSR-Colab-Fork v0.2

---


Colab adaptation of AudioSR, with some tweaks:
- added a chunking feature to process input of any length
- added stereo handling, stereo input channels will be splitted and processed independantly (dual mono) and then reconstructed as stereo audio.
- added overlap feature to smooth the transitions between chunks (**don't use high values because AudioSR is not 100% phase accurate and this will create weird phase cancellation in the overlapping regions**)

---
Adaptation & tweaks by [jarredou](https://https://github.com/jarredou/)

Original work [AudioSR: Versatile Audio Super-resolution at Scale](https://github.com/haoheliu/versatile_audio_super_resolution) by Haohe Liu, Ke Chen, Qiao Tian, Wenwu Wang, Mark D. Plumbley



In [ ]:
#@markdown # Installation
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/haoheliu/versatile_audio_super_resolution.git
%cd versatile_audio_super_resolution
!pip install -r requirements.txt
!pip install cog huggingface_hub unidecode phonemizer einops torchlibrosa transformers ftfy timm librosa



### **IMPORTANT NOTE**

#### If the inference cell crashes, restart the runtime (do not disconnect, just restart it), else it will cause memory errors !

*If you're are doing multiple runs, think also to restart the runtime every 4 or 5 files to clean up memory*

In [ ]:
#@markdown #Inference
input_file_path = '/content/versatile_audio_super_resolution/test_LR_30s.wav' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
#@markdown ---
ddim_steps= 50 #@param {type:"slider", min:20, max:200, step:10}
overlap = 0.04 #@param {type:"slider", min:0, max:0.96, step:0.04}
guidance_scale=3.5 #@param {type:"slider", min:1, max:15, step:0.5}
seed = 0 # @param {type:"integer"}
chunk_size = 10.24 # @param ["5.12", "10.24"] {type:"raw"}
%cd /content/versatile_audio_super_resolution
import gc
import os
import random
import numpy as np
from scipy.signal.windows import hann
import soundfile as sf
import torch
from cog import BasePredictor, Input, Path
import tempfile
import librosa
from audiosr import build_model, super_resolution

os.environ["TOKENIZERS_PARALLELISM"] = "true"
torch.set_float32_matmul_precision("high")

class Predictor(BasePredictor):
    def setup(self, model_name="basic", device="auto"):
        self.model_name = model_name
        self.device = device
        self.sr = 48000
        print("Loading Model...")
        self.audiosr = build_model(model_name=self.model_name, device=self.device)
        print("Model loaded !")

    def process_audio(self, input_file, chunk_size=5.12, overlap=0.1, seed=None, guidance_scale=3.5, ddim_steps=50):

        # check if input_file is a file path or a numpy array
        if isinstance(input_file, str):
            audio, sr = sf.read(input_file)
        else:
            audio = input_file
        print(f"original samplerate = {sr}")
        # check if audio is stereo & split channels
        is_stereo = len(audio.shape) == 2
        if is_stereo:
            print("audio is stereo")
            audio_ch1, audio_ch2 = audio[:, 0], audio[:, 1]
        else:
            print("Audio is mono")
            audio_ch1 = audio


        # define chunk and overlap size in samples based on input sample rate
        chunk_samples = int(chunk_size * sr)
        overlap_samples = int(overlap * chunk_samples)

        # calculate chunk size and overlap based on output sample rate
        output_chunk_samples = int(chunk_size * self.sr)
        output_overlap_samples = int(overlap * output_chunk_samples)
        enable_overlap = True if overlap > 0 else False

        print(f"enable_overlap = {enable_overlap}")
        def process_chunks(audio):
            chunks = []
            original_lengths = []
            start = 0
            while start < len(audio):
                end = min(start + chunk_samples, len(audio))
                chunk = audio[start:end]
                if len(chunk) < chunk_samples:
                    original_lengths.append(len(chunk))
                    pad = np.zeros(chunk_samples - len(chunk))
                    chunk = np.concatenate([chunk, pad])
                else:
                    original_lengths.append(chunk_samples)
                chunks.append(chunk)
                start += chunk_samples - overlap_samples if enable_overlap else chunk_samples
            return chunks, original_lengths

        # create chunks lists for each channel
        chunks_ch1, original_lengths_ch1 = process_chunks(audio_ch1)
        if is_stereo:
            chunks_ch2, original_lengths_ch2 = process_chunks(audio_ch2)

        # process each chunk with the model and reconstruct the audio
        sample_rate_ratio = self.sr / sr
        total_length = len(chunks_ch1) * output_chunk_samples - (len(chunks_ch1) - 1) * (output_overlap_samples if enable_overlap else 0)
        reconstructed_ch1 = np.zeros((1, total_length))
        # print(reconstructed_ch1.shape)

        for i, chunk in enumerate(chunks_ch1):
            print(f"Processing chunk {i+1} of {len(chunks_ch1)} for Left/Mono channel")
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as temp_wav:
                sf.write(temp_wav.name, chunk, sr)
                # print(f"chunk.shape = {chunk.shape}")

                out_chunk = super_resolution(
                    self.audiosr,
                    temp_wav.name,
                    seed=seed,
                    guidance_scale=guidance_scale,
                    ddim_steps=ddim_steps,
                    latent_t_per_second=12.8
                )
                # remove all junk added by audiosr
                # print(f"out_chunk.shape = {out_chunk.shape}")
                out_chunk = out_chunk[0]
                # print(f"reshaped out_chunk.shape = {out_chunk.shape}")
                num_samples_to_keep = int(original_lengths_ch1[i] * sample_rate_ratio)
                # print(f"num_samples_to_keep : {num_samples_to_keep}")
                out_chunk = out_chunk[:, :num_samples_to_keep]


                # apply crossfade if overlap is enabled
                if enable_overlap:
                    # calculate the actual overlap size for this chunk
                    actual_overlap_samples = min(output_overlap_samples, num_samples_to_keep)

                    # create fade-out and fade-in arrays of the correct size
                    fade_out = np.linspace(1., 0., actual_overlap_samples)
                    fade_in = np.linspace(0., 1., actual_overlap_samples)

                    if i == 0:
                        out_chunk[:, -actual_overlap_samples:] *= fade_out

                    elif i < len(chunks_ch1) - 1:
                        out_chunk[:, :actual_overlap_samples] *= fade_in
                        out_chunk[:, -actual_overlap_samples:] *= fade_out

                    else:
                        out_chunk[:, :actual_overlap_samples] *= fade_in

                # print(f"out_chunk.shape : {out_chunk.shape}")

                start = i * (output_chunk_samples - output_overlap_samples if enable_overlap else output_chunk_samples)
                end = start + out_chunk.shape[1]
                reconstructed_ch1[0, start:end] += out_chunk.flatten()


        if is_stereo:
            reconstructed_ch2 = np.zeros((1, total_length))
            # print(reconstructed_ch2.shape)

            for i, chunk in enumerate(chunks_ch2):
                print(f"Processing chunk {i+1} of {len(chunks_ch2)} for Right channel")
                with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as temp_wav:
                    sf.write(temp_wav.name, chunk, sr)
                    # print(f"chunk.shape = {chunk.shape}")

                    out_chunk = super_resolution(
                        self.audiosr,
                        temp_wav.name,
                        seed=seed,
                        guidance_scale=guidance_scale,
                        ddim_steps=ddim_steps,
                        latent_t_per_second=12.8
                    )

                    # remove all junk added by audiosr
                    # print(f"out_chunk.shape = {out_chunk.shape}")
                    out_chunk = out_chunk[0]
                    # print(f"reshaped out_chunk.shape = {out_chunk.shape}")
                    num_samples_to_keep = int(original_lengths_ch2[i] * sample_rate_ratio)
                    # print(f"num_samples_to_keep : {num_samples_to_keep}")
                    out_chunk = out_chunk[:, :num_samples_to_keep]

                    # apply crossfade if overlap is enabled
                    if enable_overlap:
                        # calculate the actual overlap size for this chunk
                        actual_overlap_samples = min(output_overlap_samples, num_samples_to_keep)

                        # create fade-out and fade-in arrays of the correct size
                        fade_out = np.linspace(1., 0., actual_overlap_samples)
                        fade_in = np.linspace(0., 1., actual_overlap_samples)

                        # no fadein for 1st chunk
                        if i == 0:
                            out_chunk[:, -actual_overlap_samples:] *= fade_out

                        elif i < len(chunks_ch1) - 1:
                            out_chunk[:, :actual_overlap_samples] *= fade_in
                            out_chunk[:, -actual_overlap_samples:] *= fade_out

                        # no fadeout for last  chunk
                        else:
                            out_chunk[:, :actual_overlap_samples] *= fade_in

                    # print(f"out_chunk.shape : {out_chunk.shape}")

                    start = i * (output_chunk_samples - output_overlap_samples if enable_overlap else output_chunk_samples)
                    end = start + out_chunk.shape[1]
                    reconstructed_ch2[0, start:end] += out_chunk.flatten()

                reconstructed_audio = np.stack([reconstructed_ch1, reconstructed_ch2], axis=-1)
        else:
            reconstructed_audio = reconstructed_ch1

        # print(f"reconstructed_audio shape : {reconstructed_audio.shape}")
        return reconstructed_audio[0]

    def predict(self,
        input_file: Path = Input(description="Audio to upsample"),
        ddim_steps: int = Input(description="Number of inference steps", default=50, ge=10, le=500),
        guidance_scale: float = Input(description="Scale for classifier free guidance", default=3.5, ge=1.0, le=20.0),
        overlap: float = Input(description="overlap size", default=0.1),
        chunk_size: float = Input(description="chunksize", default=5.12),
        seed: int = Input(description="Random seed. Leave blank to randomize the seed", default=None)
    ) -> Path:

        if seed == 0:
            seed = random.randint(0, 2**32 - 1)
        print(f"Setting seed to: {seed}")
        print(f"overlap = {overlap}")
        print(f"guidance_scale = {guidance_scale}")
        print(f"ddim_steps = {ddim_steps}")
        print(f"chunk_size = {chunk_size}")
        print(f"input file = {os.path.basename(input_file)}")

        waveform = self.process_audio(
            input_file,
            chunk_size=chunk_size,
            overlap=overlap,
            seed=seed,
            guidance_scale=guidance_scale,
            ddim_steps=ddim_steps
        )
        os.makedirs(output_folder, exist_ok=True)
        sf.write(f"{output_folder}/SR_{os.path.basename(input_file)}", data=waveform, samplerate=48000,  subtype="FLOAT")
        print(f"file created: {output_folder}/SR_{os.path.basename(input_file)}")
        del self.audiosr
        gc.collect()
        torch.cuda.empty_cache()


if __name__ == "__main__":
    p = Predictor()
    p.setup()
    out = p.predict(
        input_file_path,
        ddim_steps=ddim_steps,
        guidance_scale=guidance_scale,
        seed=seed,
        chunk_size=chunk_size,
        overlap=overlap
    )

    del p
    gc.collect()
    torch.cuda.empty_cache()
